In [9]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Debug
import ipdb
deb = ipdb.set_trace

# Imports
import os
import sys
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go 

# Personal imports
sys.path.append("{}/../../../analysis_code/utils".format(os.getcwd()))
from plot_utils import plotly_template

In [4]:
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'

In [34]:
# Load general settings
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
tasks = analysis_info['task_intertask'][0]
prf_task_name = analysis_info['prf_task_name']

In [35]:
# General figure settings
template_specs = dict(axes_color="rgba(0, 0, 0, 1)",
                      axes_width=2,
                      axes_font_size=15,
                      bg_col="rgba(255, 255, 255, 1)",
                      font='Arial',
                      title_font_size=15,
                      plot_width=1.5)

fig_template = plotly_template(template_specs)

colormap_subject_dict = {'sub-01': '#AA0DFE', 
                         'sub-02': '#3283FE', 
                         'sub-03': '#85660D', 
                         'sub-04': '#782AB6', 
                         'sub-05': '#565656', 
                         'sub-06': '#1C8356', 
                         'sub-07': '#16FF32', 
                         'sub-08': '#F7E1A0', 
                         'sub-09': '#E2E2E2', 
                         'sub-11': '#1CBE4F', 
                         'sub-12': '#C4451C', 
                         'sub-13': '#DEA0FD', 
                         'sub-14': '#FBE426', 
                         'sub-15': '#325A9B', 
                         'sub-16': '#FEAF16', 
                         'sub-17': '#F8A19F', 
                         'sub-18': '#90AD1C', 
                         'sub-20': '#F6222E', 
                         'sub-21': '#1CFFCE', 
                         'sub-22': '#2ED9FF', 
                         'sub-23': '#B10DA1', 
                         'sub-24': '#C075A6', 
                         'sub-25': '#FC1CBF'}

In [59]:
perf_thr_froup = {}
for task in tasks:
    perf_thr_froup[task] = [] 
    if task == prf_task_name:
        subjects = [s for s in analysis_info['subjects'] if s != 'sub-01']
    else:
        subjects = analysis_info['subjects']
    base_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
    task_settings_path = os.path.join(base_dir, project_dir, '{}_settings.json'.format(task))
    with open(task_settings_path) as f:
        settings = json.load(f)
    
    eye = settings['eye']
    num_run = settings['num_run']
    num_seq = settings['num_seq']
    seq_trs = settings['seq_trs']
    eye_mov_seq = settings['eye_mov_seq']
    trials_seq = settings['trials_seq']
    rads = settings['rads']
    pursuits_tr = np.arange(0,seq_trs,2)
    saccades_tr = np.arange(1,seq_trs,2)
    eyetracking_sampling = settings['eyetrack_sampling']
    screen_size = settings['screen_size']
    ppd = settings['ppd']
    
    threshold = settings['threshold']
    precision_thr = 0.7
           
    precision_data = {}
    n_sub_sup_thr = 0 
    n_sub_tot = 0 
    for subject_to_group in subjects:
        # Defind directories
        eye_tracking_dir = '{}/{}/derivatives/pp_data/{}/eyetracking'.format(
            main_dir, project_dir, subject_to_group)
        
        # Lad data
        precision_summary = pd.read_csv("{}/stats/{}_task-{}_precision_summary.tsv".format(
            eye_tracking_dir, subject_to_group, task), delimiter='\t')
        precision_one_summary = pd.read_csv("{}/stats/{}_task-{}_precision_one_threshold_summary.tsv".format(
            eye_tracking_dir, subject_to_group, task), delimiter='\t')
        
        #Concat subjects
        precision_data[subject_to_group] = {
            "precision_mean": precision_summary["precision_mean"],
            "precision_one_thrs_mean": precision_one_summary["precision_one_thrs_mean"].item()
        }
        
    sorted_precision_data = dict(sorted(precision_data.items(),
                                        key=lambda item: item[1]["precision_one_thrs_mean"],
                                        reverse=True))
    
    thresholds=np.linspace(0, 9, 100)
    for subject_to_group, data in sorted_precision_data.items():
        idx = np.where(thresholds == threshold)[0]
        perf_thr_froup[task].append(data["precision_mean"][idx[0]])
        if data["precision_mean"][idx[0]] > precision_thr:
            n_sub_sup_thr += 1
        n_sub_tot += 1
    
    task_median = np.median(perf_thr_froup[task])
    task_ci_down = np.percentile(perf_thr_froup[task], 5)
    task_ci_up = np.percentile(perf_thr_froup[task], 95)
    
    print('Their is {}/{} subject over {}% of performance at {} dva for {} task'.format(n_sub_sup_thr, n_sub_tot, precision_thr*100, threshold, task))
    print('{} task median is {:.2f}% with [{:.2f}% - {:.2f}%] confidence interval\n'.format(task, task_median*100, task_ci_down*100, task_ci_up*100))
        

     


Their is 18/20 subject over 70.0% of performance at 2 dva for PurLoc task
PurLoc task median is 90.31% with [65.90% - 98.24%] confidence interval

Their is 16/20 subject over 70.0% of performance at 2 dva for SacLoc task
SacLoc task median is 87.33% with [45.70% - 95.47%] confidence interval

Their is 19/19 subject over 70.0% of performance at 2 dva for pRF task
pRF task median is 95.67% with [81.65% - 99.49%] confidence interval



In [50]:
np.percentile(perf_thr_froup['PurLoc'], 0.05)

0.34392610731511253